# ADC Simulationn (Linearität)
Version: 12.2020

Mit dieser Simulation sollen grundsätzlich RunUp und RunDown Funktionen

# Library laden


In [1]:
import SimADC_V1 as adc
import pandas as pd
import plotly.graph_objects as px

import plotly.io as pio
pio.renderers.default = 'iframe'


## Simulation starten


## Diagramm erstellen
Vout am Integrator

Beipiele und Dokumention finden sie auf der folgenden Seite:
https://plotly.com/python/<br>
https://plotly.com/python/bar-charts/#basic-bar-chart-with-plotlygraphobjects

In [2]:
# Ausgabe 100 Werten

x_vals = []
y_vals= []
integrator = adc.Integrator()
integrator.Vin = 5.0


integrator.VrefN = -7.0448
integrator.VrefP =  7.0448
integrator.ChargingInjection = 100.0 # pC 

integrator.R_VrefN = 20.00  # [kOhm]
integrator.R_VrefP = 20.0  # [kOhm]
integrator.R_Vin =   40.00  # [kOhm]
integrator.C = 2.2 # in nF



cpu = adc.CPU()


# CPU Ticks = 0.2 us => 5 Mhz
cpu.RunUpCycles = 6250   # 
cpu.RunUpCnt    = 50    #  50 * 0.2 = 10us
cpu.WaveForm    = 0   # Zwischenkompensation

# Auflösung: 6250 * 30  = 1.875.000
# Bei Vin +/-10V = 20V / Aufslösung = 10 uV (theoretisch) 


cpu.Connect( integrator) # Kontrolle für den Integrator an die CPU übergeben
cpu.ResidueADC = adc.ResidueADC(cpu, 12, 3.0) # Reste ADC mit n bit erzeugen



# Kalibration Array 
# Parameter: Vadc Vin gemessen ADC
# Parameter: Vtrue Vin real

#-----------------------------------
# Waveform = 10
if cpu.WaveForm == 0:
    calibration_points = [
        (-5.8343680055 , -6.0),  # -Vref
        (0.0, 0.0),            # 0 V
        (5.8343680055  ,   6.0)    # +Vref
    ]

if cpu.WaveForm == 2:
    calibration_points = [
        (-6.502763102 , -6.0),  # -Vref
        (0.0, 0.0),            # 0 V
        (6.502763102  ,   6.0)    # +Vref
    ]




# Initialisiere die Kalibrierung
cpu.Calibration = adc.ADCCalibration(calibration_points)





integrator.Vin = -5.0
for a in range(0,200+1):
    
    cpu.Reset()
    
    #cpu.AutoZero()
    cpu.RunUpV2()
    cpu.RunDown()
    cpu.CalcVin() # Berechnet den Wert der CPU - Vin 
    

    
    delta = (integrator.Vin - cpu.Vin) 
    x_vals.append( integrator.Vin)
    y_vals.append( delta)
    
    print("delta  : ", delta , " V")
    
    integrator.Vin = integrator.Vin + 0.05
    
    
    
    
    

delta  :  4.381897058269857e-05  V
delta  :  -5.91025839762338e-05  V
delta  :  -3.056722679062318e-05  V
delta  :  -2.0318696032362027e-06  V
delta  :  2.6503487585038954e-05  V
delta  :  5.503884477242593e-05  V
delta  :  -4.7882709788282796e-05  V
delta  :  -1.934735260000764e-05  V
delta  :  9.18800458649116e-06  V
delta  :  3.772336177298996e-05  V
delta  :  -6.625871896392965e-05  V
delta  :  -3.772336177743085e-05  V
delta  :  -9.188004590043874e-06  V
delta  :  1.9347352597343104e-05  V
delta  :  4.788270978561826e-05  V
delta  :  -5.503884477420229e-05  V
delta  :  -2.650348758681531e-05  V
delta  :  2.031869599683489e-06  V
delta  :  3.0567226787958646e-05  V
delta  :  5.9102583974457445e-05  V
delta  :  -4.3818970585363104e-05  V
delta  :  -1.5283613398420215e-05  V
delta  :  1.3251743789854942e-05  V
delta  :  4.178710097635374e-05  V
delta  :  -6.113445358346681e-05  V
delta  :  -3.259909639652392e-05  V
delta  :  -4.063739210025119e-06  V
delta  :  2.4471617978250038e-05 

In [3]:

#print(x_vals)


# neues Diagramm erstellen mit Titel
fig = px.Figure()


# Linie anzeigen
fig.add_trace( px.Scatter( x=x_vals, y=y_vals , name="Vout" )  )

fig.update_layout(
   title_text='Abweichung der Messung',
    yaxis_title="Delta in [V]",
    xaxis_title="Vin in [V]",
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)



fig.show()


